In [133]:
class BabyNames:
    
    def __init__(self):
        self.folder = None
        self.data = None
    
    def Constructor(self, file_location = None):
        
        import pandas as pd
#         states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
#                   "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
#                   "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
#                   "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
#                   "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
        
        # Test states
        states = ["AL", "AK", "AZ", "WI", "WY"]
        
        df = pd.DataFrame()
        
        for i in states:
            temp = pd.read_csv(file_location + "/" + str("AL") + ".TXT", sep=",", header = None)
            df = pd.concat([df, temp])

        df = df.rename(columns={0: "state", 1: "sex", 2: "year", 3: "name", 4: "births"})
        
        self.data = df
        self.folder = file_location
        return self
    
    
    def Count(self, state = "", year = ""):
        
        return self
    
    def Top10BabyNames(self, state = "IL", year = 2015):
        
        return self
    
    def ChangeOfPopularity(self, fromYear = 2014, toYear = 2015, top = 10):
        
        return self    
    
    def Top5NamesPerYear(self, year = 2015, sex = ""):
        
        return self
    
    def NamePopularityPlot(name = "Jim", yearRange = (2000, 2015), state = "IL", sex = "M"):
        
        return self
    
    def NameFlip(self, n = 10):
        
        return self

In [134]:
baby_test = BabyNames().Constructor(file_location = "babynames")

In [139]:
baby_test.data

,state,sex,year,name,births
0,AL,F,1910,Mary,875
1,AL,F,1910,Annie,482
2,AL,F,1910,Willie,257
3,AL,F,1910,Mattie,232
4,AL,F,1910,Ruby,204
...,...,...,...,...,...
139288,AL,M,2021,Zealand,5
139289,AL,M,2021,Zechariah,5
139290,AL,M,2021,Zeke,5
139291,AL,M,2021,Ziaire,5
